In [1]:
import os
import re
import pandas as pd
import numpy as np
import json
import alpaca_trade_api as tradeapi
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LinearRegression

In [17]:
from datetime import datetime

In [2]:
os.chdir('../')
print(os.getcwd())

c:\Users\pehbo\projects\algo


In [3]:
# Replace with your own API key and secret
with open('secrets/secrets.json') as f:
    secrets = json.load(f)
API_KEY = secrets['KEY']
API_SECRET = secrets['SECRET']
BASE_URL = 'https://paper-api.alpaca.markets'  # For paper trading, use the paper trading URL

# Set up the Alpaca API client
api = tradeapi.REST(API_KEY, API_SECRET, base_url=BASE_URL, api_version='v2')

In [4]:
symbol = 'TSLA'
start_date = '2010-01-01'
end_date = '2023-03-25'

historical_data = api.get_bars(symbol, tradeapi.rest.TimeFrame.Day, start=start_date, end=end_date).df
historical_data.index.name = 'date'
historical_data.reset_index(inplace=True)

In [9]:
# Download the TSLA news data
tsla_news = api.get_news(symbol, start_date, end_date)
news_summaries = [news.summary for news in tsla_news]

In [12]:
# Extract significant words and phrases using CountVectorizer
vectorizer = CountVectorizer(max_features=50)
X = vectorizer.fit_transform(news_summaries).toarray()

# Create a DataFrame to store significant words and phrases
significant_words = pd.DataFrame(X, columns=vectorizer.get_feature_names_out())

# Merge the TSLA historical data and significant_words DataFrames
historical_data['date'] = pd.to_datetime(historical_data['date']).dt.date


In [22]:
news_dates = [news.updated_at.date() for news in tsla_news]
significant_words['date'] = news_dates
merged_data = pd.merge(historical_data, significant_words, on='date', how='inner')

# Train a linear regression model to predict the next closing price
X_train = merged_data.drop(['date', 'close'], axis=1)
y_train = merged_data['close']

lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Predict the next closing price
X_test = X_train.iloc[-1].values.reshape(1, -1)
next_closing_price = lr_model.predict(X_test)

print("Predicted next closing price for TSLA:", next_closing_price[0])

Predicted next closing price for TSLA: 190.4100000000035


c:\Users\pehbo\anaconda3\envs\algo\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
